In [1]:
import pandas as pd
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
import seaborn as sns

reading the datasets from database

In [2]:
train_df = pd.read_csv("apartments_100k.csv", sep=";", encoding='latin1')
test_df = pd.read_csv("appartments_10k.csv",  sep=";", encoding='latin1')


/tmp/ipython-input-1229441265.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("apartments_100k.csv", sep=";", encoding='latin1')


basic info of the dataset

In [3]:
print('Shape of the train data file',train_df.shape)
print('Shape of the test data file',test_df.shape)

Shape of the train data file (99492, 22)
Shape of the test data file (10000, 22)


cheacking features with there number of columns and rows

In [4]:
def count_categorical_numerical_columns(df)-> Tuple[int, int]:
    categorical_columns = df.select_dtypes(include=['object']).columns
    num_categorical_columns = len(categorical_columns)
    numerical_columns = df.select_dtypes(exclude=['object']).columns
    num_numerical_columns = len(numerical_columns)
    return num_categorical_columns, num_numerical_columns

num_categorical, num_numerical = count_categorical_numerical_columns(train_df)
print("Number of categorical columns:", num_categorical)
print("Number of numerical columns:", num_numerical)

Number of categorical columns: 14
Number of numerical columns: 8


joining the  columns into data

In [5]:

data = pd.concat([train_df, test_df], ignore_index=True)

data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [6]:
data.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_type,square_feet,address,cityname,state,latitude,longitude,source,time,"time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
0,5509055389,housing/rent/apartment,"Lovely Suwanee, Two BR, Two BA","Square footage: 1076 sq-ft, unit number: 1511....","AC,Cable or Satellite,Clubhouse,Dishwasher,Ele...",2.0,2.0,USD,No,Yes,...,Monthly,1076,NaN,Suwanee,GA,34.0648,-84.0849,RentDigs.com,1.568771e+09,NaN
1,5198602576,housing/rent/apartment,Apartment in prime location,"Square footage: 965 sq-ft, unit number: 0905. ...","AC,Cable or Satellite,Clubhouse,Gated,Gym,Inte...",1.0,1.0,USD,No,Thumbnail,...,Monthly,965,NaN,Shreveport,LA,32.4544,-93.7135,RentDigs.com,1.550863e+09,NaN
2,5668639190,housing/rent/apartment,Studio apartment 1475 W. Adams Blvd.,"This unit is located at 1475 W. Adams Blvd., L...",NaN,1.0,0.0,USD,No,Thumbnail,...,Monthly,600,1475 W Adams Boulevard,Los Angeles,CA,34.0372,-118.2972,RentLingo,1.577360e+09,NaN
3,5668620791,housing/rent/apartment,"One BR 2032 Tx 361,","This unit is located at 2032 Tx 361,, Inglesid...","Cable or Satellite,Clubhouse,Dishwasher,Gated,...",1.0,1.0,USD,No,Thumbnail,...,Monthly,722,2032 TX 361,Ingleside,TX,27.8689,-97.2057,RentLingo,1.577359e+09,NaN
4,5509228509,housing/rent/apartment,"Average Rent $1,275 a month - That's a STEAL!",If you think you can t afford to live on a 36-...,"Parking,Pool,Tennis",1.0,2.0,USD,No,Thumbnail,...,Monthly,960,NaN,East Hartford,CT,41.7588,-72.6148,RentDigs.com,1.568780e+09,NaN


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109492 entries, 0 to 109491
Data columns (total 23 columns):
 #   Column                                                                      Non-Null Count   Dtype  
---  ------                                                                      --------------   -----  
 0   id                                                                          109492 non-null  object 
 1   category                                                                    109491 non-null  object 
 2   title                                                                       109492 non-null  object 
 3   body                                                                        109492 non-null  object 
 4   amenities                                                                   89900 non-null   object 
 5   bathrooms                                                                   109395 non-null  object 
 6   bedrooms                            

In [8]:
def strip_spaces(df):
    df.columns = df.columns.str.strip()
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()

    return df
df=strip_spaces(data)

In [9]:
print(df.duplicated().sum())
print(df.shape)

84
(109492, 23)


removing duplicates from the data set

In [10]:
df=df.drop_duplicates()
df.shape

(109408, 23)

In [11]:
df=df.drop(columns=['id','time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,','title','body'])

data.head()

In [12]:
df.head()

,category,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,housing/rent/apartment,"AC,Cable or Satellite,Clubhouse,Dishwasher,Ele...",2.0,2.0,USD,No,Yes,"Cats,Dogs",1920.0,"$1,920",Monthly,1076,nan,Suwanee,GA,34.0648,-84.0849,RentDigs.com,1.568771e+09
1,housing/rent/apartment,"AC,Cable or Satellite,Clubhouse,Gated,Gym,Inte...",1.0,1.0,USD,No,Thumbnail,"Cats,Dogs",973.0,$973,Monthly,965,nan,Shreveport,LA,32.4544,-93.7135,RentDigs.com,1.550863e+09
2,housing/rent/apartment,nan,1.0,0.0,USD,No,Thumbnail,"Cats,Dogs",1585.0,"$1,585",Monthly,600,1475 W Adams Boulevard,Los Angeles,CA,34.0372,-118.2972,RentLingo,1.577360e+09
3,housing/rent/apartment,"Cable or Satellite,Clubhouse,Dishwasher,Gated,...",1.0,1.0,USD,No,Thumbnail,"Cats,Dogs",950.0,$950,Monthly,722,2032 TX 361,Ingleside,TX,27.8689,-97.2057,RentLingo,1.577359e+09
4,housing/rent/apartment,"Parking,Pool,Tennis",1.0,2.0,USD,No,Thumbnail,nan,1275.0,"$1,275",Monthly,960,nan,East Hartford,CT,41.7588,-72.6148,RentDigs.com,1.568780e+09


In [13]:
df['category'].value_counts()

,count
category,
housing/rent/apartment,109297
housing/rent/commercial/retail,42
housing/rent,7
"Parking"",Storage,Washer Dryer,Wood Floors",7
housing/rent/short_term,6
housing/rent/home,6
Parking,5
"Cable or Satellite"",Internet Access,Refrigerator",3
housing/rent/condo,3


as the catagrory column is huge imbalnaced so we have to remove that to

In [14]:
df = df.drop(columns=['category'])

In [15]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 109408 entries, 0 to 109491
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   amenities      109408 non-null  object 
 1   bathrooms      109408 non-null  object 
 2   bedrooms       109408 non-null  object 
 3   currency       109408 non-null  object 
 4   fee            109408 non-null  object 
 5   has_photo      109408 non-null  object 
 6   pets_allowed   109408 non-null  object 
 7   price          109407 non-null  float64
 8   price_display  109408 non-null  object 
 9   price_type     109408 non-null  object 
 10  square_feet    109408 non-null  object 
 11  address        109408 non-null  object 
 12  cityname       109408 non-null  object 
 13  state          109408 non-null  object 
 14  latitude       109408 non-null  object 
 15  longitude      109327 non-null  float64
 16  source         109408 non-null  object 
 17  time           99408 non-null   fl

In [16]:
df['amenities'].isnull().sum()


np.int64(0)

In [17]:
df['amenities'].value_counts()

,count
amenities,
nan,19582
Parking,6404
"Parking,Storage",2181
"Gym,Pool",1925
Pool,1654
...,...
"Clubhouse,Gym,Patio/Deck,Storage,TV,Washer Dryer",1
"AC,Dishwasher,Gym,Patio/Deck,Pool,Refrigerator,Washer Dryer",1
"Cable or Satellite,Dishwasher,Elevator,Fireplace,Hot Tub,Parking,Patio/Deck,Pool,Storage,Tennis",1


In [18]:
df['bathrooms'].value_counts()

,count
bathrooms,
1.0,56262
2.0,36538
1,6615
1.5,3449
2,2397
2.5,2394
3.0,889
3.5,273
3,174


In [19]:
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')


In [20]:
df['bathrooms'].value_counts()


,count
bathrooms,
1.0,62877
2.0,38935
1.5,3449
2.5,2394
3.0,1063
3.5,273
4.0,166
4.5,54
5.0,28


In [21]:

df['bedrooms']  = pd.to_numeric(df['bedrooms'], errors='coerce')
df['latitude']  = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')


In [22]:
df['currency'].value_counts()

,count
currency,
USD,109362
nan,24
"Cats,Dogs",20
"Cats"",Dogs",2


In [23]:
df['currency'].dropna(inplace=True)

In [24]:
df['fee'].value_counts()

,count
fee,
No,109161
Yes,201
1350,5
1470,2
2495,2
3796,1
1810,1
5000,1
1380,1


In [25]:

df['fee'] = df['fee'].apply(lambda x: 0 if x == 'No' else 1)


In [26]:
df['fee'].value_counts()

,count
fee,
0,109161
1,247


In [27]:
df['has_photo'].value_counts()

,count
has_photo,
Yes,56795
Thumbnail,43186
No,9381
"$1,350",5
"$1,470",2
"$2"",495",2
"$3,796",1
"$1,810",1
"$5,000",1


In [28]:
df['has_photo'] = df['has_photo'].apply(lambda x: 0 if x == 'Yes' or x=='Thumbnail' else 1)

In [29]:
df['has_photo'].value_counts()

,count
has_photo,
0,99981
1,9427


In [30]:
df['pets_allowed'].value_counts()

,count
pets_allowed,
nan,64513
"Cats,Dogs",42270
Cats,2325
Dogs,251
Monthly,46
"Cats"",Dogs",2
"Cats,Dogs,None",1


In [31]:
df['pets_allowed']=df['pets_allowed'].str.replace('nan', 'no')
df['pets_allowed']=df['pets_allowed'].str.replace('Cats,Dogs', 'yes')
df['pets_allowed']=df['pets_allowed'].str.replace('Cats",Dogs', 'yes')
df['pets_allowed']=df['pets_allowed'].str.replace('Cats,Dogs,None	', 'yes')
df['pets_allowed']=df['pets_allowed'].str.replace('Monthly', 'yes')
df['pets_allowed']=df['pets_allowed'].str.replace('yes,None', 'yes')



In [32]:
df['pets_allowed'].value_counts()

,count
pets_allowed,
no,64513
yes,42319
Cats,2325
Dogs,251


In [33]:
df['price'].isnull().sum()

np.int64(1)

In [34]:
df['price'].fillna(df['price'].mean(), inplace=True)

/tmp/ipython-input-1085838478.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(df['price'].mean(), inplace=True)


In [35]:
df['price_display'].value_counts()


,count
price_display,
"$1,350",655
$850,626
"$1,200",606
"$1,100",599
$950,596
...,...
"$4,290",1
"$3,472",1
"$2,696",1


In [36]:
df['price_display'].isnull().sum()

np.int64(0)

In [37]:
df['price_type'].value_counts()

,count
price_type,
Monthly,109356
Norman,7
Weekly,4
Dallas,3
Monthly|Weekly,2
Morrisville,2
Los Angeles,2
Las Vegas,2
Windermere,2


In [38]:
df['price_type'] = 'Monthly'


In [39]:
df=df.drop(columns=['price_type'])

In [40]:
df['square_feet'].value_counts()

,count
square_feet,
1000,1612
700,1542
900,1533
800,1515
750,1379
...,...
3560,1
2899,1
2618,1


In [41]:
def categorize_square_feet(df, col='square_feet'):
    df[col] = pd.to_numeric(df[col], errors='coerce')
    def categorize(x):
        if pd.isna(x):
            return -1
        elif x <= 500:
            return 0
        elif x <= 700:
            return 1
        elif x <= 900:
            return 2
        elif x <= 1100:
            return 3
        elif x <= 1300:
            return 4
        elif x <= 1500:
            return 5
        else:
            return 6

    df['square_feet'] = df[col].apply(categorize)
    return df
df=categorize_square_feet(df)

In [42]:
df['square_feet'].value_counts()

,count
square_feet,
2,31802
3,24564
1,19429
4,17046
5,6104
6,5586
0,4831
-1,46


In [43]:
df['square_feet']=df['square_feet'].apply(lambda x: 0 if x == -1 else x)
df['square_feet'].value_counts()



,count
square_feet,
2,31802
3,24564
1,19429
4,17046
5,6104
6,5586
0,4877


In [44]:
df['address'].value_counts()

,count
address,
nan,94751
8215 S.W 72nd Avenue,19
2647 Eastgate Road,9
35.2505,7
901 Valley View Circle,7
...,...
207 Russell St Middletown Court 06457-Plex,1
1443 Euclid Avenue,1
321 W Laurel,1


as city name is given and most the data in address is nan so we simply do one step which is droping that column becaue it is just waste to ful full or pridict that columns

In [45]:
df=df.drop(columns=['address'])


In [46]:
print (df['cityname'].value_counts())

cityname
Dallas           3069
Denver           2855
Los Angeles      2592
Las Vegas        1925
Arlington        1666
                 ... 
Stanardsville       1
Osseo               1
Los Banos           1
Algonac             1
Lihue               1
Name: count, Length: 3016, dtype: int64


as there is 3016 cities we keep few that are most common cities and on the basis of them we should do one thing which is mention other city in city so that over model dont do over fitting there

In [47]:
print (sum(df['cityname'].value_counts()>500))
df['cityname'].value_counts() > 500

28


,count
cityname,
Dallas,True
Denver,True
Los Angeles,True
Las Vegas,True
Arlington,True
...,...
Stanardsville,False
Osseo,False
Los Banos,False


In [48]:

top_cities = df['cityname'].value_counts()[df['cityname'].value_counts() > 500].index
df['cityname'] = df['cityname'].apply(lambda x: x if x in top_cities else 'Other')


In [49]:
df['cityname'].value_counts()

,count
cityname,
Other,78419
Dallas,3069
Denver,2855
Los Angeles,2592
Las Vegas,1925
Arlington,1666
Atlanta,1553
Charlotte,1169
Austin,1058


In [50]:
df['state'].value_counts()

,count
state,
TX,12981
CA,11252
VA,8483
NC,6728
CO,6597
FL,6109
MD,5693
OH,5219
MA,5188


In [51]:
invalid_states = ['RentDigs.com', 'ListedBuy', 'RentLingo', 'RENTOCULAR', 'Listanza', 'nan', 'None']

df['state'] = df['state'].astype(str).str.strip()
df['state'] = df['state'].replace(invalid_states, 'Unknown')

df['state'] = df['state'].replace('nan', 'Unknown')


In [52]:
top_20_states = df['state'].value_counts().nlargest(20).index
top_20_states


Index(['TX', 'CA', 'VA', 'NC', 'CO', 'FL', 'MD', 'OH', 'MA', 'GA', 'NJ', 'WA',
       'NV', 'AZ', 'MO', 'LA', 'IL', 'PA', 'TN', 'NE'],
      dtype='object', name='state')

In [53]:
df['state'] = df['state'].apply(lambda x: x if x in top_20_states else 'Other')


In [54]:
df['state'].value_counts()

,count
state,
Other,14862
TX,12981
CA,11252
VA,8483
NC,6728
CO,6597
FL,6109
MD,5693
OH,5219


In [55]:
df['latitude'].describe()

,latitude
count,109327.000000
mean,37.016215
std,4.694450
min,19.573800
25%,33.746200
50%,37.338200
75%,40.035100
max,64.833200


In [56]:
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')


In [57]:
df['longitude'].value_counts()

,count
longitude,
-97.7497,1026
-71.0710,567
-74.0644,535
-77.0132,490
-84.3922,376
...,...
-89.0627,1
-71.5141,1
-82.3139,1


In [58]:
df['longitude'].describe()

,longitude
count,109327.000000
mean,-91.849562
std,15.837557
min,-159.369800
25%,-104.781100
50%,-85.909900
75%,-78.456700
max,-68.778800


In [59]:
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['longitude'].fillna(df['longitude'].median(), inplace=True)


/tmp/ipython-input-2597882184.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['longitude'].fillna(df['longitude'].median(), inplace=True)


In [60]:
df['source'].value_counts()

,count
source,
RentDigs.com,93552
RentLingo,13835
ListedBuy,747
GoSection8,468
RealRentals,337
tenantcloud,125
BostonApartments,94
Listanza,51
nan,46


In [61]:
df=df.drop(columns=['source'])

this feauture is just telling us like from where it is getting the the data so this means this column is not good or use full for price prediction data set so i am just dropping that columns or feature what you supposed to do

In [62]:
df=df.drop(columns=['time'])

Reason 1 — The model learns noise

Date posted has no meaningful effect on rent.

Reason 2 — Old data

Listings are from 2018–2020 → outdated, no inflation adjustment.

Reason 3 — Many duplicates in timestamps

Listing sites batch-post uploads → same timestamp for many rows.

Reason 4 — It becomes a useless random feature

Hurts tree models
Hurts linear regression
Hurts neural networks

In [63]:
def categorize_bathrooms(df, col='bathrooms'):
    df[col] = pd.to_numeric(df[col], errors='coerce')

    def convert(b):
        if b <= 1:
            return 1
        elif b <= 2:
            return 2
        elif b <= 3:
            return 3
        elif b <= 4:
            return 4
        else:
            return 5

    df['bathrooms'] = df[col].apply(convert)
    return df
df=categorize_bathrooms(df)

In [65]:
def categorize_bathrooms(df, col='bedrooms'):
    df[col] = pd.to_numeric(df[col], errors='coerce')

    def convert(b):
        if b <= 1:
            return 1
        elif b <= 2:
            return 2
        elif b <= 3:
            return 3
        elif b <= 4:
            return 4
        elif b <= 5:
            return 5
        elif b <= 6:
            return 6
        else:
            return 7

    df['bedrooms'] = df[col].apply(convert)
    return df
df=categorize_bathrooms(df)

In [66]:
df.to_csv("cleaned_apartments.csv", index=False)


In [67]:
df.head()

,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price,price_display,square_feet,cityname,state,latitude,longitude
0,"AC,Cable or Satellite,Clubhouse,Dishwasher,Ele...",2,2,USD,0,0,yes,1920.0,"$1,920",3,Other,GA,34.0648,-84.0849
1,"AC,Cable or Satellite,Clubhouse,Gated,Gym,Inte...",1,1,USD,0,0,yes,973.0,$973,3,Other,LA,32.4544,-93.7135
2,nan,1,1,USD,0,0,yes,1585.0,"$1,585",1,Los Angeles,CA,34.0372,-118.2972
3,"Cable or Satellite,Clubhouse,Dishwasher,Gated,...",1,1,USD,0,0,yes,950.0,$950,2,Other,TX,27.8689,-97.2057
4,"Parking,Pool,Tennis",1,2,USD,0,0,no,1275.0,"$1,275",3,Other,Other,41.7588,-72.6148
